# 1. The Investigator

Target
- a phone number

Conditions
-  a number that spelled something using the letters printed on the phone buttons: like 2 has “ABC”, and 3 “DEF”, etc.

In [1]:
import pandas as pd
NC = pd.read_csv('noahs-customers.csv')
NC.head()

,customerid,name,address,citystatezip,birthdate,phone,timezone,lat,long
0,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468
1,1002,Julie Howell,185-1 Linden St,"Brooklyn, NY 11221",1956-12-03,680-537-8725,America/New_York,40.69426,-73.921670
2,1003,Christopher Ali,174-28 Baisley Blvd,"Jamaica, NY 11434",2001-09-20,315-846-6054,America/New_York,40.68902,-73.773470
3,1004,Christopher Rodriguez,102 Mount Hope Pl,"Bronx, NY 10453",1959-07-10,516-275-2292,America/New_York,40.84939,-73.909160
4,1005,Jeffrey Wilkinson,17 St Marks Pl,"Manhattan, NY 10003",1988-09-08,838-830-6960,America/New_York,40.72804,-73.987120


In [2]:
phone2 = NC['phone'].str.replace('-','')
NC['phone2'] = phone2
phone2_idx = NC.columns.get_loc("phone2")
phone_idx = NC.columns.get_loc("phone")
cols = NC.columns.tolist()
cols = cols[:phone_idx + 1] + cols[phone2_idx:phone2_idx + 1] + cols[phone_idx + 1:phone2_idx]
NC = NC[cols]
NC.head()

,customerid,name,address,citystatezip,birthdate,phone,phone2,timezone,lat,long
0,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,3153775031,America/New_York,40.71817,-73.997468
1,1002,Julie Howell,185-1 Linden St,"Brooklyn, NY 11221",1956-12-03,680-537-8725,6805378725,America/New_York,40.69426,-73.921670
2,1003,Christopher Ali,174-28 Baisley Blvd,"Jamaica, NY 11434",2001-09-20,315-846-6054,3158466054,America/New_York,40.68902,-73.773470
3,1004,Christopher Rodriguez,102 Mount Hope Pl,"Bronx, NY 10453",1959-07-10,516-275-2292,5162752292,America/New_York,40.84939,-73.909160
4,1005,Jeffrey Wilkinson,17 St Marks Pl,"Manhattan, NY 10003",1988-09-08,838-830-6960,8388306960,America/New_York,40.72804,-73.987120


In [3]:
fullnames = NC['name'].str.split(expand=True) # Expand split strs into separate cols
midl, last = fullnames[1], fullnames[2]
last = last.mask( last.isnull(), midl )
NC['last'] = last
name_idx = NC.columns.get_loc("name")
last_idx = NC.columns.get_loc("last")
cols = NC.columns.tolist()
cols = cols[:name_idx + 1] + cols[last_idx:last_idx + 1] + cols[name_idx + 1:last_idx]
NC = NC[cols]
NC.head()

,customerid,name,last,address,citystatezip,birthdate,phone,phone2,timezone,lat,long
0,1001,Jacqueline Alvarez,Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,3153775031,America/New_York,40.71817,-73.997468
1,1002,Julie Howell,Howell,185-1 Linden St,"Brooklyn, NY 11221",1956-12-03,680-537-8725,6805378725,America/New_York,40.69426,-73.921670
2,1003,Christopher Ali,Ali,174-28 Baisley Blvd,"Jamaica, NY 11434",2001-09-20,315-846-6054,3158466054,America/New_York,40.68902,-73.773470
3,1004,Christopher Rodriguez,Rodriguez,102 Mount Hope Pl,"Bronx, NY 10453",1959-07-10,516-275-2292,5162752292,America/New_York,40.84939,-73.909160
4,1005,Jeffrey Wilkinson,Wilkinson,17 St Marks Pl,"Manhattan, NY 10003",1988-09-08,838-830-6960,8388306960,America/New_York,40.72804,-73.987120


In [6]:
NC = NC[['last','phone2']]
NC.head()

,last,phone2
0,Alvarez,3153775031
1,Howell,6805378725
2,Ali,3158466054
3,Rodriguez,5162752292
4,Wilkinson,8388306960


In [14]:
def last_name_to_phone_number (last):
    D={'abc':2,'def':3,'ghi':4,'jkl':5,'mno':6,'pqrs':7,'tuv':8,'wxyz':9}
    res = ''
    for char in last:
        for k,v in D.items():
            if char.lower() in k:
                res += str(v)
                break
    return res
NC['to_check'] = NC['last'].apply( last_name_to_phone_number )
NC.head()

,last,phone2,to_check
0,Alvarez,3153775031,2582739
1,Howell,6805378725,469355
2,Ali,3158466054,254
3,Rodriguez,5162752292,763744839
4,Wilkinson,8388306960,945546766


In [15]:
NC[ NC['phone2'] == NC['to_check'] ]

,last,phone2,to_check
207,Tannenbaum,8266362286,8266362286
